In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression, RFECV, RFE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance

In [2]:
# Download the Energy Efficiency Dataset from UCI ML Repository
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
df = pd.read_excel(url)
# Separate the features (X) and target variable (y)
X = df.iloc[:, :-2]  # Select all columns except the last two (features)
y = df.iloc[:, -1:]  # Select the label from the last two columns (target variables)

In [3]:
X.shape

(768, 8)

In [4]:
X.dtypes

X1    float64
X2    float64
X3    float64
X4    float64
X5    float64
X6      int64
X7    float64
X8      int64
dtype: object

In [5]:
# standardizing
X = (X - X.mean())/X.std(ddof=1)
feature_names = X.columns

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## using Sklearn RFE 

In [47]:
from sklearn.feature_selection import RFECV

def perform_rfe(X_train, y_train, X_test, model):
    # Create an RFE object with 5-fold cross-validation
    rfe = RFECV(estimator=model, cv=5, verbose=0, scoring='neg_mean_absolute_percentage_error')

    # Perform feature selection on the training data
    X_train_selected = rfe.fit_transform(X_train, y_train)
    
    print("no of features selected:", rfe.n_features_)
    print("X selected: ", X_train_selected.shape)
    # Get the support of the selected features
    support = rfe.support_
    print(rfe.support_)
    # Get the names of the selected features
    selected_features = [feature_names[i] for i in range(len(feature_names)) if support[i]]
    
    cv_results = rfe.cv_results_
    mean_test_scores = cv_results['mean_test_score']
    best_idx = np.argmax(mean_test_scores)
    print("mean CV scores:", mean_test_scores)
    
    # Apply feature selection to the testing data
    X_test_selected = rfe.transform(X_test)

    #print("ranks for features. lower is better.")
    #print(rfe.ranking_)
    return X_train_selected, X_test_selected, selected_features


In [48]:
# Create models
linear_model = LinearRegression()
# Perform RFE with Linear Regression model
X_train_rfe_linear, X_test_rfe_linear, selected_features_rfe_linear = perform_rfe(X_train, y_train, X_test, linear_model)

no of features selected: 6
X selected:  (614, 6)
[ True  True  True  True  True False  True False]
mean CV scores: [-0.13384525 -0.12867851 -0.12880245 -0.11787789 -0.1066102  -0.08950637
 -0.09044526 -0.09053393]


In [49]:
# observe mean CV scores, the scores are for best 1,2,3...8 feature models. 6 features option has the highest -MAPE.

In [50]:
# Perform RFE with Decision Tree model
tree_model = DecisionTreeRegressor()
X_train_rfe_tree, X_test_rfe_tree, selected_features_rfe_tree = perform_rfe(X_train, y_train, X_test, tree_model)

no of features selected: 8
X selected:  (614, 8)
[ True  True  True  True  True  True  True  True]
mean CV scores: [-0.11768132 -0.08611914 -0.03904611 -0.03785958 -0.04035149 -0.03785365
 -0.0375911  -0.03686154]


In [51]:
# observe mean CV scores, the scores are for best 1,2,3...8 feature models. 8 features option has the highest -MAPE.

In [9]:
# sorted(sklearn.metrics.SCORERS.keys()) shows scoring options

In [10]:
selected_features_rfe_linear

['X1', 'X2', 'X3', 'X4', 'X5', 'X7']

In [11]:
X_train_rfe_linear.shape

(614, 6)

In [12]:
selected_features_rfe_tree

['X1', 'X2', 'X3', 'X6', 'X7', 'X8']

## using SelectKBest

In [13]:
# Create the SelectKBest object
k = 5  # Number of top features to select
selector = SelectKBest(score_func=f_regression, k=k)

# Fit the selector to the training data
selector.fit(X_train, y_train)

# Get the selected features
selected_features = selector.get_support(indices=True)
selected_feature_names = [feature_names[i] for i in selected_features]

# Transform the training and testing data to keep only the selected features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Train a linear regression model on the selected features
model = LinearRegression()
model.fit(X_train_selected, y_train)

# Evaluate the model on the testing data
score = model.score(X_test_selected, y_test)

# Print the selected features and the model score
print("Selected Features:", selected_feature_names)
print("Model Score:", score)


Selected Features: ['X1', 'X2', 'X3', 'X4', 'X5']
Model Score: 0.8541115688411918


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## using RFE (without CV)

In [14]:
estimator = LinearRegression()
rfe = RFE(estimator=estimator, n_features_to_select=None)

# Evaluate model performance for different numbers of features
scores = []
num_features = range(1, X.shape[1] + 1)

for k in num_features:
    rfe.n_features_to_select = k
    X_selected = rfe.fit_transform(X, y)
    model = LinearRegression()
    model.fit(X_selected, y)
    y_pred = model.predict(X_selected)
    score = r2_score(y, y_pred)
    scores.append(score)

# Find the optimal number of features based on the highest score
optimal_num_features = num_features[np.argmax(scores)]

# Use the optimal_num_features for feature selection
rfe.n_features_to_select = optimal_num_features
X_selected = rfe.fit_transform(X, y)

In [20]:
print("no of features:",len(scores))
print("r square for best models with varying no of features:", scores)
print("based on highest r square, best no of features:", optimal_num_features)

no of features: 8
r square for best models with varying no of features: [0.7439866432524689, 0.7774655257789087, 0.7719628145944, 0.822680504627051, 0.8435885321091069, 0.8872078371928117, 0.8869802289997921, 0.8876871197455591]
based on highest r square, best no of features: 8


#### r square always increases with feature count. so it is not a good way to compare models with different no of features. modify this code to get MAPE or RMSE for each model. or use adjusted r square.

## using mlxtend SFS (forward selection and backward elimination)

In [15]:
# Create the linear regression model
model = LinearRegression()

# Create the SequentialFeatureSelector object
sfs = SequentialFeatureSelector(estimator=model, k_features='best', forward=True, verbose=2, scoring='r2')

# Perform feature selection
sfs.fit(X_train, y_train)

# Get the selected feature indices
selected_feature_indices = sfs.k_feature_idx_

# Get the selected feature names
selected_feature_names = [feature_names[i] for i in selected_feature_indices]

# Transform the training and testing data to keep only the selected features
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

# Train a linear regression model on the selected features
model.fit(X_train_selected, y_train)

# Evaluate the model on the testing data
score = model.score(X_test_selected, y_test)

# Print the selected features and the model score
print("Selected Features:", selected_feature_names)
print("Model Score:", score)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s finished

[2023-05-29 10:47:52] Features: 1/8 -- score: 0.8000636362172259[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s finished

[2023-05-29 10:47:52] Features: 2/8 -- score: 0.842959316574895[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s finished

[2023-05-29 10:47:52] Features: 3/8 -- score: 0.8772456518805024[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 o

Selected Features: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7']
Model Score: 0.8931291583109783


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-05-29 10:47:53] Features: 6/8 -- score: 0.8838408781438556[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-05-29 10:47:53] Features: 7/8 -- score: 0.8838463795204088[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished

[2023-05-29 10:47:53] Features: 8/8 -- score: 0.8820719695504179

## Permutation importance

In [16]:
# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Perform permutation importance for Linear Regression
linear_results = permutation_importance(linear_model, X_test, y_test, n_repeats=10, random_state=42)
linear_importances = linear_results.importances_mean

# Sort the linear feature importances in descending order
linear_sorted_indices = linear_importances.argsort()[::-1]
linear_sorted_feature_names = feature_names[linear_sorted_indices]
linear_sorted_importances = linear_importances[linear_sorted_indices]

# Print the sorted linear feature importances
print("Linear Regression Feature Importances:")
for feature, importance in zip(linear_sorted_feature_names, linear_sorted_importances):
    print(f"{feature}: {importance}")

# Decision Tree Regression
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)

# Perform permutation importance for Decision Tree Regression
tree_results = permutation_importance(tree_model, X_test, y_test, n_repeats=10, random_state=42)
tree_importances = tree_results.importances_mean

# Sort the decision tree feature importances in descending order
tree_sorted_indices = tree_importances.argsort()[::-1]
tree_sorted_feature_names = feature_names[tree_sorted_indices]
tree_sorted_importances = tree_importances[tree_sorted_indices]

# Print the sorted decision tree feature importances
print("\nDecision Tree Regression Feature Importances:")
for feature, importance in zip(tree_sorted_feature_names, tree_sorted_importances):
    print(f"{feature}: {importance}")


Linear Regression Feature Importances:
X1: 1.2513473800880708
X5: 1.0951274284940549
X4: 0.37920500791286105
X2: 0.3747715850219111
X7: 0.07377439196330729
X3: 0.0005505470273164282
X6: 0.0005379491454088669
X8: 0.00021772242474873415

Decision Tree Regression Feature Importances:
X5: 2.2309198781474633
X2: 0.2616295694683056
X7: 0.07453719066251532
X1: 0.03644309018380656
X8: 0.03196278188739378
X3: 0.02502556819797357
X6: 0.01880653559814697
X4: 0.0001316841934896118
